In [1]:
from pathlib import Path
import polars as pl

In [2]:
ROOT_DIR = Path("/home/bobby/repos/latent-neural-dynamics-modeling")
saved_dir = ROOT_DIR / "data"

In [3]:
participants = pl.read_parquet(
    saved_dir / "resampled" / "participants_intermediate.parquet"
)

In [4]:
PDI_labels = pl.read_csv(saved_dir / "PDI_labels.csv")
PDI_labels_trials = pl.read_csv(saved_dir / "PDI_labels_trials.csv")
PDI_trial_durations = pl.read_csv(saved_dir / "PDI_trial_durations.csv")

In [5]:
PDI_labels_trials = PDI_labels_trials.select(
    pl.col("subject").alias("participant_id"),
    pl.col("session").cast(pl.UInt32),
    pl.col("ix_block").cast(pl.UInt32).alias("block"),
    pl.col("ix_trial").cast(pl.UInt32).alias("trial"),
    pl.when(pl.col("dbs_stim") == "on").then(1).otherwise(0).alias("dbs_stim"),
    pl.col("yscore").cast(pl.Float32),
)

PDI_labels_trials = PDI_labels_trials.sort(
    ["participant_id", "session", "block", "trial"]
)

PDI_labels_trials = (
    PDI_labels_trials.group_by(
        ["participant_id", "session", "block"], maintain_order=True
    )
    .agg(
        pl.col("trial").alias("trials"),
        pl.col("dbs_stim"),
        pl.col("yscore").alias("yscores"),
    )
    .with_columns(pl.col("trials").list.len().alias("trial_count"))
)

In [6]:
PDI_labels = PDI_labels.select(
    pl.col("subject").alias("participant_id"),
    pl.col("session").cast(pl.UInt32),
    pl.col("ix_block").cast(pl.UInt32).alias("block"),
    pl.when(pl.col("dbs_stim") == "on").then(1).otherwise(0).alias("dbs_stim"),
    pl.col("valid_trial_cnt").cast(pl.UInt32),
)

PDI_labels = PDI_labels.sort(["participant_id", "session", "block"])

In [7]:
PDI_trial_durations = PDI_trial_durations.select(
    pl.col("sub").str.split(by="-").list.get(-1).alias("participant_id"),
    pl.col("ses").str.split(by="-").list.get(-1).cast(pl.UInt32).alias("session"),
    pl.col("block").cast(pl.UInt32),
    pl.col("dt_s").cast(pl.Float32),
).sort(["participant_id", "session", "block"])

In [8]:
participants = (
    participants.with_columns(pl.col("participant_id").str.split(by="-").list.get(-1))
    .with_columns(
        pl.col("ieeg_headers_file")
        .str.split("/")
        .list.get(-1)
        .str.split(".")
        .list.get(0)
        .alias("base_ieeg_file")
    )
    .with_columns(
        pl.concat_str(
            pl.lit(str(saved_dir / "resampled")),
            pl.concat_str(pl.col("base_ieeg_file"), pl.lit("parquet"), separator="."),
            separator="/",
        ).alias("ieeg_parquet"),
        pl.col("session").cast(pl.UInt32),
        pl.col("run").cast(pl.UInt32).alias("block"),
        pl.col("session_path")
        .str.split(by="/")
        .list.tail(2)
        .list.join(separator="/")
        .map_elements(lambda s: str(saved_dir.joinpath(s)), return_dtype=pl.String)
        .alias("session_path"),
    )
    .drop("base_ieeg_file", "ieeg_headers_file", "durations", "run")
)

participants = participants.drop("participant_path", "ieeg_path", "ieeg_file").unique()

In [9]:
PDI_trial_durations = PDI_trial_durations.join(
    PDI_labels, on=["participant_id", "session", "block"], how="left"
)
PDI_trial_durations = PDI_trial_durations.join(
    PDI_labels_trials, on=["participant_id", "session", "block"], how="left"
)

In [10]:
PDI_trial_durations = PDI_trial_durations.with_columns(
    pl.col("valid_trial_cnt").fill_null(-1), pl.col("trial_count").fill_null(-1)
)

In [11]:
# PDI4-3-12 all trials valid
# PDI3-4-12 all good for classification but won't take the x and y coordinates
PDI_trial_durations.filter(pl.col("valid_trial_cnt") != pl.col("trial_count"))

participant_id,session,block,dt_s,dbs_stim,valid_trial_cnt,trials,dbs_stim_right,yscores,trial_count
str,u32,u32,f32,i32,i64,list[u32],list[i32],list[f32],i64
"""PDI3""",4,12,9.018765,1,11,null,null,null,-1
"""PDI4""",3,12,9.002478,1,12,null,null,null,-1


In [12]:
"""
Process the PDI_trial_durations DataFrame according to the following logic:
    - For any row where trial_count is -1 but valid_trial_cnt is not -1, set the 'trials'
        column to the list [1, 2, ..., 12]. (This indicates there are no pre-assigned trial labels.)
    - For rows where both trial_count and valid_trial_cnt are -1, remove these rows from
        the DataFrame.
The function prints all the rows that were removed.
"""

"\nProcess the PDI_trial_durations DataFrame according to the following logic:\n    - For any row where trial_count is -1 but valid_trial_cnt is not -1, set the 'trials'\n        column to the list [1, 2, ..., 12]. (This indicates there are no pre-assigned trial labels.)\n    - For rows where both trial_count and valid_trial_cnt are -1, remove these rows from\n        the DataFrame.\nThe function prints all the rows that were removed.\n"

In [13]:
removed_rows = PDI_trial_durations.filter(
    (pl.col("trial_count") == -1) & (pl.col("valid_trial_cnt") == -1)
)
print("Removed rows:")
print(
    removed_rows.select(
        "participant_id", "session", "block", "valid_trial_cnt", "trial_count"
    )
)

Removed rows:
shape: (4, 5)
┌────────────────┬─────────┬───────┬─────────────────┬─────────────┐
│ participant_id ┆ session ┆ block ┆ valid_trial_cnt ┆ trial_count │
│ ---            ┆ ---     ┆ ---   ┆ ---             ┆ ---         │
│ str            ┆ u32     ┆ u32   ┆ i64             ┆ i64         │
╞════════════════╪═════════╪═══════╪═════════════════╪═════════════╡
│ PDI3           ┆ 4       ┆ 1     ┆ -1              ┆ -1          │
│ PDI3           ┆ 4       ┆ 2     ┆ -1              ┆ -1          │
│ PDI3           ┆ 4       ┆ 3     ┆ -1              ┆ -1          │
│ PDI4           ┆ 3       ┆ 9     ┆ -1              ┆ -1          │
└────────────────┴─────────┴───────┴─────────────────┴─────────────┘


In [14]:
# Exclude the rows where both trial_count and valid_trial_cnt are -1
PDI_trial_durations = PDI_trial_durations.filter(
    ~((pl.col("trial_count") == -1) & (pl.col("valid_trial_cnt") == -1))
)

In [15]:
PDI_trial_durations = PDI_trial_durations.with_columns(
    [
        pl.when(pl.col("trial_count") == -1)
        .then(pl.lit(list(range(1, 13))))
        .otherwise(pl.col("trials"))
        .alias("trials"),
        pl.when(pl.col("trial_count") == -1)
        .then(pl.col("valid_trial_cnt").eq(12))
        .otherwise(pl.col("trial_count") == pl.col("valid_trial_cnt"))
        .alias("labels"),
    ]
)

In [16]:
PDI_trial_durations.filter(~pl.col("labels"))

participant_id,session,block,dt_s,dbs_stim,valid_trial_cnt,trials,dbs_stim_right,yscores,trial_count,labels
str,u32,u32,f32,i32,i64,list[u32],list[i32],list[f32],i64,bool
"""PDI3""",4,12,9.018765,1,11,"[1, 2, … 12]",null,null,-1,false


In [17]:
PDI_trial_durations = PDI_trial_durations.drop(
    "valid_trial_cnt", "trial_count", "dbs_stim_right"
)

In [18]:
participants = participants.join(
    PDI_trial_durations, on=["participant_id", "session", "block"], how="left"
)

In [19]:
participants.filter(pl.col("dbs_stim").is_null())

participant_id,session_path,session,onsets,ieeg_parquet,block,dt_s,dbs_stim,trials,yscores,labels
str,str,u32,list[f64],str,u32,f32,i32,list[u32],list[f32],bool
"""PDI3""","""/home/bobby/repos/latent-neura…",4,"[12.227955, 29.381682, … 199.263364]","""/home/bobby/repos/latent-neura…",3,null,null,null,null,null
"""PDI4""","""/home/bobby/repos/latent-neura…",3,"[10.723909, 25.000409, … 190.073364]","""/home/bobby/repos/latent-neura…",9,null,null,null,null,null
"""PDI3""","""/home/bobby/repos/latent-neura…",4,"[12.854636, 30.407727, … 268.204773]","""/home/bobby/repos/latent-neura…",2,null,null,null,null,null
"""PDI3""","""/home/bobby/repos/latent-neura…",4,"[62.370591, 84.879727, … 269.472682]","""/home/bobby/repos/latent-neura…",1,null,null,null,null,null


In [20]:
participants = participants.filter(pl.col("dbs_stim").is_not_null())

In [21]:
participants.write_parquet(
    saved_dir / "participants.parquet",
    partition_by=["participant_id", "session"],
)

In [22]:
participants = pl.read_parquet(
    "/home/bobby/repos/latent-neural-dynamics-modeling/resampled_recordings/participants/participant_id=PDI1/session=2/*"
)

In [23]:
participants

participant_id,session,block,trial,onset,margined_onset,margined_duration,time,time_original,motion_time,original_length_ts,start_ts,trial_length_ts,chunk_margin,dbs_stim,yscore,LFP_1,LFP_2,LFP_3,LFP_4,LFP_5,LFP_6,LFP_7,LFP_8,LFP_9,LFP_10,LFP_11,LFP_12,LFP_13,LFP_14,LFP_15,LFP_16,ECOG_1,ECOG_2,ECOG_3,ECOG_4,x,y,x_interpolated,y_interpolated,tracing_coordinates_present
str,u32,u32,u32,f64,f64,f32,list[f64],list[f64],list[f64],u32,u32,u32,i32,i32,f32,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[i64],list[i64],list[f32],list[f32],bool
"""PDI1""",2,1,1,18.102864,16.102864,13.019526,"[16.102864, 16.103864, … 29.120864]","[18.102864, 18.103864, … 27.120864]","[18.102864, 18.111301, … 27.113953]",9019,13019,13019,2,0,2.703554,"[-0.00001, -0.000014, … 0.000003]","[-0.000004, -0.000006, … 0.000006]","[0.000002, 0.000008, … 0.000003]","[0.000015, -0.000063, … -0.000079]","[-8.7940e-7, -0.000006, … 0.000004]","[-0.000014, 0.00001, … -0.000014]","[-0.000038, 0.000016, … -0.000044]","[-0.000002, -0.000004, … 0.000006]","[-0.000002, -0.000004, … 0.000007]","[0.00001, 0.00001, … 0.00001]","[-0.000002, -0.000002, … 0.000006]","[-9.6913e-7, -0.000002, … 0.000003]","[0.000002, 0.000002, … 0.000004]","[-0.000007, -0.000007, … 0.000003]","[-0.000015, -0.000015, … -0.000002]","[-0.000001, -8.7139e-7, … 0.000001]","[0.000004, 0.000006, … 0.000023]","[0.000004, 0.000006, … 0.000016]","[0.000008, 0.00001, … 0.000006]","[0.000004, 0.000007, … -0.000006]","[-377, -387, … -23]","[37, 48, … -164]","[-377.0, -378.184296, … -23.0]","[37.0, 38.302727, … -164.0]",true
"""PDI1""",2,1,2,36.896455,34.896455,13.019526,"[34.896455, 34.897455, … 47.914455]","[36.896455, 36.897455, … 45.914455]","[36.896455, 36.904956, … 45.90748]",9019,13019,13019,2,0,1.811423,"[-0.00001, -0.000014, … 0.000003]","[-0.000004, -0.000006, … 0.000006]","[0.000002, 0.000008, … 0.000003]","[0.000015, -0.000063, … -0.000079]","[-8.7940e-7, -0.000006, … 0.000004]","[-0.000014, 0.00001, … -0.000014]","[-0.000038, 0.000016, … -0.000044]","[-0.000002, -0.000004, … 0.000006]","[-0.000002, -0.000004, … 0.000007]","[0.00001, 0.00001, … 0.00001]","[-0.000002, -0.000002, … 0.000006]","[-9.6913e-7, -0.000002, … 0.000003]","[0.000002, 0.000002, … 0.000004]","[-0.000007, -0.000007, … 0.000003]","[-0.000015, -0.000015, … -0.000002]","[-0.000001, -8.7139e-7, … 0.000001]","[0.000004, 0.000006, … 0.000023]","[0.000004, 0.000006, … 0.000016]","[0.000008, 0.00001, … 0.000006]","[0.000004, 0.000007, … -0.000006]","[-377, -390, … 122]","[37, 37, … 47]","[-377.0, -378.528046, … 122.0]","[37.0, 37.0, … 47.0]",true
"""PDI1""",2,1,3,55.729091,53.729091,13.019526,"[53.729091, 53.730091, … 66.747091]","[55.729091, 55.730091, … 64.747091]","[55.729091, 55.73756, … 64.740148]",9019,13019,13019,2,0,2.518298,"[-0.00001, -0.000014, … 0.000003]","[-0.000004, -0.000006, … 0.000006]","[0.000002, 0.000008, … 0.000003]","[0.000015, -0.000063, … -0.000079]","[-8.7940e-7, -0.000006, … 0.000004]","[-0.000014, 0.00001, … -0.000014]","[-0.000038, 0.000016, … -0.000044]","[-0.000002, -0.000004, … 0.000006]","[-0.000002, -0.000004, … 0.000007]","[0.00001, 0.00001, … 0.00001]","[-0.000002, -0.000002, … 0.000006]","[-9.6913e-7, -0.000002, … 0.000003]","[0.000002, 0.000002, … 0.000004]","[-0.000007, -0.000007, … 0.000003]","[-0.000015, -0.000015, … -0.000002]","[-0.000001, -8.7139e-7, … 0.000001]","[0.000004, 0.000006, … 0.000023]","[0.000004, 0.000006, … 0.000016]","[0.000008, 0.00001, … 0.000006]","[0.000004, 0.000007, … -0.000006]","[-377, -382, … 107]","[37, 50, … 29]","[-377.0, -377.589935, … 107.0]","[37.0, 38.533821, … 29.0]",true
"""PDI1""",2,1,4,72.946727,70.946727,13.019526,"[70.946727, 70.947727, … 83.964727]","[72.946727, 72.947727, … 81.964727]","[72.946727, 72.955244, … 81.957737]",9019,13019,13019,2,0,2.052713,"[-0.00001, -0.000014, … 0.000003]","[-0.000004, -0

In [24]:
df = pl.read_parquet(
    "/home/bobby/repos/latent-neural-dynamics-modeling/data/PDI_behav_features.parquet"
)

In [25]:
df

trial_time,pre_trial_time,is_fragmented,delta_t,speed,acceleration,velocity_x,velocity_y,acceleration_x,acceleration_y,isj_x,isj_y,isj,speed_sub,acceleration_sub,velocity_x_sub,velocity_y_sub,acceleration_x_sub,acceleration_y_sub,isj_x_sub,isj_y_sub,isj_sub,pathlen,dt,dt_l,dist_sub,dt_norm,len,speed_sub_bin_1,accel_sub_bin_1,jerk_sub_bin_1,dist_sub_bin_1,speed_sub_bin_2,accel_sub_bin_2,jerk_sub_bin_2,dist_sub_bin_2,speed_sub_bin_3,accel_sub_bin_3,jerk_sub_bin_3,dist_sub_bin_3,speed_sub_bin_4,accel_sub_bin_4,jerk_sub_bin_4,dist_sub_bin_4,speed_sub_bin_5,accel_sub_bin_5,jerk_sub_bin_5,dist_sub_bin_5,speed_sub_bin_6,accel_sub_bin_6,jerk_sub_bin_6,dist_sub_bin_6,speed_sub_bin_7,accel_sub_bin_7,jerk_sub_bin_7,dist_sub_bin_7,speed_sub_bin_8,accel_sub_bin_8,jerk_sub_bin_8,dist_sub_bin_8,file,subject,session,ix_block,ix_trial,stim,stim_ampl,stim_unit,exclude_due_to_protocol
f64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,i64,i64,str,str,str,bool
9.0,2.06486,false,0.008419,410.837504,91821.078717,142.444444,352.666667,31924.82716,78630.888889,36831.0,196740.0,116785.5,407.175488,4112.749537,141.623924,350.899056,1809.076291,3253.702747,1990.0,5234.0,3612.0,1068,15767.369865,15767.369865,9.536891,14.749644,2.043977,5.12,0.02,-0.06,10.783,7.795455,-0.155172,-0.267442,9.024766,4.241379,0.086207,0.137931,11.228896,-1.0,0.625,0.375,8.870217,-4.153846,0.269231,-0.076923,9.115025,-8.5,-0.032258,-0.032258,8.081716,-3.089744,0.102564,0.00641,10.345023,1.151163,-0.232558,0.27907,9.565822,"""/Volumes/MDExternM2/data/copyd…","""PDI1""","""2""",11,2,"""off""","""6""","""mA""",false
9.0,1.222764,false,0.008515,377.505752,81456.580148,138.333333,317.333333,29830.888889,68391.814815,30075.0,158082.0,94078.5,371.865975,3601.791331,136.963068,313.961494,1528.235924,2852.41762,1514.0,4849.0,3181.5,1056,13694.030133,13694.030133,8.861952,12.955563,2.021033,4.733333,-0.533333,-0.25,7.556671,7.109756,0.036585,-0.140244,9.118215,3.758621,0.293103,-0.103448,10.053308,-0.5,0.5,0.5,9.659114,-3.388889,0.194444,-0.027778,9.49299,-7.5,0.139535,-0.023256,9.556393,-3.142857,-0.035714,0.228571,8.629078,1.25,0.15,-0.133333,7.962857,"""/Volumes/MDExternM2/data/copyd…","""PDI1""","""2""",11,3,"""off""","""6""","""mA""",false
9.0,1.333596,false,0.008563,353.964917,74118.243025,108.666667,308.555556,23277.703704,64383.481481,25128.0,151753.0,88440.5,349.95284,3688.394655,106.464071,307.304105,1433.206836,2991.603426,1384.0,4875.0,3129.5,1050,12261.300701,12261.300701,8.443573,11.666318,2.00956,5.636364,-0.204545,0.204545,9.521074,6.590361,-0.096386,-0.168675,8.426759,3.958333,-0.145833,0.166667,9.257507,-2.5,2.0,-1.0,8.946067,null,null,null,null,-7.423077,0.118421,0.026316,7.619868,-3.343284,-0.089552,0.089552,8.219744,1.425,-0.2875,0.05,8.7232,"""/Volumes/MDExternM2/data/copyd…","""PDI1""","""2""",11,8,"""off""","""6""","""mA""",false
9.0,1.231359,false,0.008419,374.100145,81990.612253,126.666667,317.555556,27582.839506,69880.925926,28498.0,162935.0,95716.5,369.365973,3879.993452,125.542899,314.411765,1470.972227,3135.146776,1300.0,5928.0,3614.0,1068,13276.868268,13276.868268,8.62665,12.419895,2.043977,3.903226,-0.133333,-0.034483,8.51291,7.277778,-0.145833,-0.180556,8.669787,3.560606,0.090909,0.045455,8.787711,-0.75,0.25,0.125,7.941394,-5.5625,0.3125,0.1875,8.492045,-8.257143,-0.042857,0.085714,9.004286,-3.32716,-0.049383,0.111111,8.794755,1.448276,-0.155172,0.034483,8.669316,"""/Volumes/MDExternM2/data/copyd…","""PDI1""","""2""",11,4,"""off""","""6""","""mA""",false
9.0,1.887891,false,0.008483,400.808285,87381.884257,141.888889,339.444444,31188.160494,73798.444444,36587.0,194610.0,115598.5,395.9668,3909.790511,139.757167,336.571668,1683.787504,3084.035661,1665.0,5593.0,3629.0,1060,12007.372551,12007.372551,7.755852,11.317034,2.028681,4.808824,-0.264706,0.088235,7.1241

In [30]:
df.filter(pl.col("trial_time") < 9)

trial_time,pre_trial_time,is_fragmented,delta_t,speed,acceleration,velocity_x,velocity_y,acceleration_x,acceleration_y,isj_x,isj_y,isj,speed_sub,acceleration_sub,velocity_x_sub,velocity_y_sub,acceleration_x_sub,acceleration_y_sub,isj_x_sub,isj_y_sub,isj_sub,pathlen,dt,dt_l,dist_sub,dt_norm,len,speed_sub_bin_1,accel_sub_bin_1,jerk_sub_bin_1,dist_sub_bin_1,speed_sub_bin_2,accel_sub_bin_2,jerk_sub_bin_2,dist_sub_bin_2,speed_sub_bin_3,accel_sub_bin_3,jerk_sub_bin_3,dist_sub_bin_3,speed_sub_bin_4,accel_sub_bin_4,jerk_sub_bin_4,dist_sub_bin_4,speed_sub_bin_5,accel_sub_bin_5,jerk_sub_bin_5,dist_sub_bin_5,speed_sub_bin_6,accel_sub_bin_6,jerk_sub_bin_6,dist_sub_bin_6,speed_sub_bin_7,accel_sub_bin_7,jerk_sub_bin_7,dist_sub_bin_7,speed_sub_bin_8,accel_sub_bin_8,jerk_sub_bin_8,dist_sub_bin_8,file,subject,session,ix_block,ix_trial,stim,stim_ampl,stim_unit,exclude_due_to_protocol
f64,f64,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,i64,i64,str,str,str,bool
7.5,2.658207,false,0.015723,329.01688,38470.831472,121.066667,268.4,14653.44,30960.48,54013.0,192131.0,123072.0,323.312522,1713.925475,118.8,264.533333,695.36,1337.013333,1139.0,3799.0,2469.0,476,13912.169429,13912.169429,17.40817,29.165974,0.912046,8.764706,-0.382353,-0.441176,16.276582,11.866667,-0.392857,0.923077,19.400252,8.907407,0.685185,-0.574074,16.308501,-8.166667,6.166667,0.166667,14.466881,-12.833333,-0.5,1.166667,13.648859,-8.9375,-1.125,0.5625,16.920803,-5.09375,0.34375,-0.046875,18.756972,2.394737,-0.473684,-0.210526,19.249721,"""/Volumes/MDExternM2/data/copyd…","""PDI4""","""2""",3,7,"""on""","""4""","""mA""",true
7.5,2.906746,false,0.015991,265.589539,31037.776139,130.266667,198.266667,15108.053333,23370.791111,42242.0,112928.0,77585.0,260.261347,1339.947564,127.854777,195.56603,697.399349,949.238003,1279.0,2771.0,2025.0,468,10372.124775,10372.124775,14.659505,22.115405,0.89675,6.529412,0.28125,-0.533333,14.368355,10.904762,-0.357143,-0.142857,13.433351,4.913043,-0.413043,0.195652,13.957466,-1.833333,1.5,1.4803e-16,15.219352,-10.7,2.0,-0.7,11.01187,-9.75,-1.75,0.875,14.338755,-3.605263,-0.065789,0.092105,14.542383,2.454545,-0.477273,0.022727,15.691388,"""/Volumes/MDExternM2/data/copyd…","""PDI4""","""2""",3,6,"""on""","""4""","""mA""",true
7.5,1.634271,false,0.015789,329.166665,38374.689382,95.733333,296.8,11053.777778,34672.888889,28303.0,256820.0,142561.5,323.833066,2009.710442,94.136967,293.165618,739.995341,1707.034708,1217.0,5179.0,3198.0,474,9179.241589,9179.241589,10.922186,19.324719,0.908222,7.194444,-0.75,-0.138889,11.621217,9.5375,0.2,-0.5875,11.5351,9.611111,-2.111111,0.055556,11.467246,0.0,0.0,0.75,12.987462,null,null,null,null,-12.638889,-1.111111,0.5,11.461252,-7.1375,0.576923,0.210526,11.008363,0.5,-0.178571,-0.25,7.67273,"""/Volumes/MDExternM2/data/copyd…","""PDI4""","""2""",3,10,"""on""","""4""","""mA""",true
7.5,3.432702,false,0.015924,396.714783,47671.827619,153.2,328.933333,18371.093333,39530.506667,64936.0,335340.0,200138.0,390.028626,2323.271897,150.266667,323.466667,968.515556,1886.791111,1328.0,6061.0,3694.5,470,13289.229073,13289.229073,16.067388,28.214924,0.900574,10.5,-0.035714,-1.269231,11.834065,12.791667,-0.369565,0.23913,17.430196,8.730769,0.769231,-0.5,16.586888,-3.0,3.333333,1.5,16.823861,-12.25,2.25,2.25,17.154982,-15.1875,-1.9375,0.25,15.833344,-7.189189,0.324324,1.1702e-16,15.881171,2.769231,-0.461538,0.019231,15.854221,"""/Volumes/MDExternM2/data/copyd…","""PDI4""","""2""",3,1,"""on""","""4""","""mA""",true
7.5,2.448874,false,0.015593,240.504245,28363.264938,113.866667,188.133333,13562.062222,22078.968889,31293.0,88271.0,59782.0,236.248284,1199.910009,112.067357,185.097032,635.839141,860.085981,1018.0,2006.0,1512.0,480,10610.592226,10610.592226,14.045378,22.059443,0.919694,4.75,-1.083333,-0.25,16.323413,8.651515,-0.181818,-0.060606,13.61481